In [ ]:
import pandas as pd
from pyprojroot import here
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'viridis'

from sklearn.metrics import auc, roc_curve
from colors import *

In [ ]:
TARGET = 'bed'

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))

auc_values = {}

for path in (here() / 'data/processed/matrices/prob').glob(f'{TARGET}*.csv'):
    
    TARGET = path.stem.split('-')[0]
    MODEL = path.stem.split('-')[1]
    ORIGIN = path.stem.split('-')[2]
    HPO = path.stem.split('-')[3]
    FSE = path.stem.split('-')[4]

    if MODEL=='guess':
        continue
    
    pred_name = f'{TARGET}-{MODEL}-{ORIGIN}-{HPO}-{FSE}'
    true_name = f'{TARGET}'
    
    y_true = pd.read_csv(here() / f'data/processed/true_matrices/{true_name}.csv', index_col='Datetime')
    y_pred = pd.read_csv(here() / f'data/processed/matrices/prob/{pred_name}.csv', index_col='Datetime')

    df = pd.concat([y_true, y_pred], axis=1).dropna()
    y_true = df.iloc[:,0]
    y_pred = df.iloc[:,1]

    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    _auc = auc(fpr, tpr)
    
    ax.plot(fpr, tpr, label=f"{ORIGIN} {_auc.round(2):.2f}", color=c[TARGET])
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)

    ident = [0.0, 1.0]
    ax.plot(ident,ident, ls='--', c='k', zorder=0, lw=.7)

    auc_values[pred_name] = _auc

# Step 2: Retrieve handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

# Step 3: Sort handles and labels (here, alphabetically by label)
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: int(t[0].split(' ')[0])))

# Step 4: Create the legend with the sorted handles and labels
plt.legend(handles, labels, frameon=False, loc=4)

plt.savefig(here() / f'output/plots/aucorigin-{TARGET}.png', dpi=300, bbox_inches='tight')